In [9]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten , Conv2D , Dense , Dropout
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50 
from tensorflow.keras.utils import to_categorical

In [10]:
(x_train , y_train) ,(x_test, y_test) = cifar10.load_data()

#### **using batches or smaller data for intial testing**

In [24]:
# if i used the normal training dataset the RAM will crash
# as the image in cifar10 ds 32*32 RGB ~ 3KB in memory
#increasing its size to 224×224 image: ~600KB memory (200x larger!)
# means that 50,000 * 600 ~ 30 GB huge memory usage
#so instead we will use batches  or the number of training samples or decreasing the image size


#Solution 1 : 
# batch_size = 500 this is simple but takes memory


# SOLUTION 2: Use tf.data for memory-efficient training (RECOMMENDED)
def create_data_pipeline():
    """Create memory-efficient data pipeline using tf.data"""
    
    # Create datasets from raw data
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train_raw, y_train))
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test_raw, y_test))
    
    def preprocess_fn(image, label):
        # Cast to float32 and resize
        image = tf.cast(image, tf.float32)
        image = tf.image.resize(image, (224, 224))
        # Apply ResNet50 preprocessing
        image = tf.keras.applications.resnet50.preprocess_input(image)
        return image, label
    
    # Apply preprocessing
    train_dataset = train_dataset.map(preprocess_fn, num_parallel_calls=tf.data.AUTOTUNE)
    test_dataset = test_dataset.map(preprocess_fn, num_parallel_calls=tf.data.AUTOTUNE)
    
    # Batch and prefetch
    train_dataset = train_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
    test_dataset = test_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
    
    return train_dataset, test_dataset





In [ ]:
# Store raw data before processing
x_train_raw, x_test_raw = x_train, x_test

# Create efficient data pipeline
train_dataset, test_dataset = create_data_pipeline()# returns a pair (image ,label)
#for each section -> training and testing 

In [26]:
# if i used the normal training dataset the RAM will crash 
# x_train = tf.image.resize(tf.cast(x_train , tf.float32), (224,224))
# x_test = tf.image.resize(tf.cast(x_test , tf.float32), (224,224))

#ways for pre_processing(Normalizing section)
# Convert back to numpy for preprocessing
# x_train = x_train.numpy()
# x_test = x_test.numpy()
# Use ResNet50's preprocessing function instead of simple normalization
# This applies the correct normalization that ResNet50 expects
# x_train = ResNet50.preprocess_input(x_train)
# x_test = ResNet50.preprocess_input(x_test)

y_train = to_categorical(y_train , 10)
y_test = to_categorical(y_test , 10)

In [27]:
base_model = ResNet50(weights='imagenet' , include_top=False , input_shape=(224, 224,3))
base_model.trainable = False 

x = Flatten()(base_model.output)
x = Dense(1000, activation='relu')(x)
x = Dropout(0.5)(x)  
predictions = Dense(10, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)


# to prevent lr error , it had been made in a variable the passed to the compiling process
optimizer = tf.optimizers.Adam( learning_rate = 0.001)
model.compile(optimizer = optimizer ,
              loss = 'categorical_crossentropy' , 
              metrics= ['accuracy'])



model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_4[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 123,950,722 (472.83 MB)

 Trainable params: 100,363,010 (382.85 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

### **training phase while freezing the layers before the fc layers**

In [28]:
history = model.fit(
    train_dataset,     # <- Contains (images, labels) pairs
    epochs=10,
    validation_data=test_dataset,  # <- Contains (images, labels) pairs  
    verbose=1
)
#when i aded batch size while training it didn't work

Epoch 1/10
1105/1563 ━━━━━━━━━━━━━━━━━━━━ 18:15 2s/step - accuracy: 0.7217 - loss: 6.1567

KeyboardInterrupt: 

### ***Evaluation phase***

In [ ]:
test_loss , test_accuracy = model.evaluate(x_test , y_test)
print(f"The test loss : {test_loss}")
print(f"The test accuracy : {test_accuracy}")